In [ ]:
import time
import psutil
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parámetros básicos
nb_train_samples = 1400
nb_validation_samples = 400
epochs = 40
width_shape = 224
height_shape = 224
num_classes = 10

# Nuevas rutas actualizadas
train_data_dir = 'D:\\Escritorio\\Noveno semestre\\Electiva 3\\DataSet aves Organizado\\entrenamiento'
validation_data_dir = 'D:\\Escritorio\\Noveno semestre\\Electiva 3\\DataSet aves Organizado\\validacion'

# Función para crear y entrenar el modelo
def create_and_train_vgg16_model(learning_rate, l2_regularization, batch_size):
    train_datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.05,
        height_shift_range=0.05,
        horizontal_flip=True,
        preprocessing_function=preprocess_input
    )

    valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(width_shape, height_shape),
        batch_size=batch_size,
        class_mode='categorical'
    )

    validation_generator = valid_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(width_shape, height_shape),
        batch_size=batch_size,
        class_mode='categorical'
    )

    image_input = Input(shape=(width_shape, height_shape, 3), name='image_input')
    base_model = VGG16(input_tensor=image_input, include_top=False, weights='imagenet')

    x = Flatten()(base_model.output)
    out = Dense(num_classes, activation='softmax', kernel_regularizer='l2')(x)

    model = Model(inputs=base_model.input, outputs=out)

    # Congelar todas las capas primero
    for layer in base_model.layers:
        layer.trainable = False

    # Desbloquear solo las últimas 4 capas convolucionales
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=['accuracy']
    )

    checkpoint = ModelCheckpoint(
        'best_model.keras', monitor='val_loss', verbose=1,
        save_best_only=True, mode='min'
    )

    early_stopping = EarlyStopping(
        monitor='val_loss', patience=5, verbose=1,
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6
    )

    model_history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=[checkpoint, early_stopping, reduce_lr]
    )

    return model_history

# Hiperparámetros a probar
learning_rates = [0.0001, 0.0005]
l2_regularizations = [1e-4, 1e-5]
batch_sizes = [16, 32]

best_val_accuracy = 0
best_hyperparams = {}

for learning_rate in learning_rates:
    for l2_regularization in l2_regularizations:
        for batch_size in batch_sizes:
            print(f"\nEntrenando con lr={learning_rate}, l2={l2_regularization}, batch_size={batch_size}...")
            model_history = create_and_train_vgg16_model(
                learning_rate, l2_regularization, batch_size
            )
            val_accuracy = np.max(model_history.history['val_accuracy'])

            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_hyperparams = {
                    'learning_rate': learning_rate,
                    'l2_regularization': l2_regularization,
                    'batch_size': batch_size,
                    'val_accuracy': val_accuracy
                }

print("\n--- Mejores hiperparámetros encontrados ---")
print(best_hyperparams)